In [ ]:
URLs 
PK  HOST
1   네이버검색결과
2   네이버블로그

Links
PK    LINK         Visited    Host.FK   inbound   REGDATE(수집일)  Y/N(크롤링여부)
1  /search?q=수지                1           0
2  /id/304012                   1           1
3  /id/304045                   1           1
...

Links where Y/N='N' order by PK limit 0,N 

FK => 1번 페이지가 갖는 전체 링크의 수
PR = inbound / 전체 link
전체 link수: count(inbound = 1)
count where inbound = 1 group by Host.FK => 1번 페이지로부터 inbound 그룹내 페이지 수 = 1

In [9]:
import sqlite3

In [10]:
con = sqlite3.connect(':memory:')
cur = con.cursor()

In [45]:
cur.executescript('''
    DROP TABLE IF EXISTS URLs;
    CREATE TABLE URLs(
        PK INTEGER PRIMARY KEY,
        SCHEME TEXT NOT NULL DEFAULT 'http',
        NETLOC TEXT NOT NULL,
        REGDATE TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
    );

    DROP TABLE IF EXISTS Links;
    CREATE TABLE Links(
        PK INTEGER PRIMARY KEY,
        PATH TEXT NOT NULL DEFAULT '/',
        QS TEXT,
        FK INTEGER NOT NULL,
        REF INTEGER NOT NULL,  => 이전 도메인(outbound), inbound cnfcj
        VISITED INTEGER NOT NULL DEFAULT 0,
        REGDATE TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
    );
''')

# Links 테이블에 추가할 속성: REF INTEGER NOT NULL,
# TIMESTAMP, CURRENT_TIMESTAMP

In [46]:
cur.execute('INSERT INTO URLs(NETLOC) VALUES("www.naver.com")')

In [47]:
cur.execute('SELECT * FROM URLs')
cur.fetchall()

[(1, 'http', 'www.naver.com', '2024-03-18 01:33:37')]

In [48]:
cur.execute('DELETE FROM URLs')

In [49]:
from requests.compat import urlparse
url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지'
url_seg = urlparse(url)
print(url_seg[:])

('https', 'search.naver.com', '/search.naver', '', 'where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지', '')


In [50]:
cur.execute('SELECT PK FROM URLs WHERE NETLOC=?',(url_seg.netloc,))
temp = cur.fetchone()

if not temp:
    cur.execute('INSERT INTO URLs (SCHEME, NETLOC) VALUES(?,?)', url_seg[:2])

cur.execute('SELECT PK FROM URLs WHERE NETLOC=?',(url_seg.netloc,))
PK = cur.fetchone()[0]

In [51]:
cur.execute('SELECT COUNT(PK) FROM Links WHERE FK=? AND PATH=?', (PK, url_seg.path))

# 만약에 있는 링크면 넣지 않는다.
if cur.fetchone()[0] == 0:
    cur.execute('INSERT INTO Links(PATH, QS, FK, REF) VALUES(?,?,?, 0)',
               (url_seg.path, url_seg.query, PK))
    con.commit()

# 새로추가된 REF 때문에 에러난다.
# REF = 0 시작이라는 의미

In [52]:
cur.execute('SELECT * FROM Links')
cur.fetchall()

[(1,
  '/search.naver',
  'where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지',
  1,
  0,
  '2024-03-18 01:33:38')]

In [53]:
cur.execute('''
    SELECT SCHEME, NETLOC, PATH, QS FROM URLs INNER JOIN Links 
    ON Links.FK = URLs.PK
    WHERE Links.VISITED = 0
    ORDER BY Links.REGDATE ASC
    LIMIT 0,10
''')

In [58]:
from bs4 import BeautifulSoup

In [59]:
from requests.compat import urlparse, urlunparse, urljoin #urlunparse:url다시합치기
from requests import get
import re


# 왜 무한반복하는 거지?
while True:
    # ORDER BY ASC => Queue
    # 아직 방문 안한 url들을 받아오기
    cur.execute('''
        SELECT Links.PK, Links.FK, SCHEME, NETLOC, PATH, QS FROM URLs INNER JOIN Links 
        ON Links.FK = URLs.PK
        WHERE Links.VISITED = 0
        ORDER BY Links.REGDATE ASC
        LIMIT 0,10
    ''')
    
    # Links.PK => 어느 사이트에서 왔는가
    # Urls.PK => 수집했는지 여부, 업데이트용
    # Urls.FK => 수집했는지 여부, 업데이트용
    
    urls = cur.fetchall()

    if len(urls) == 0:
        break

    # 모든 urls들이 없어질 때까지 반복하기
    while urls:
        url_seg = urls.pop(0)
        print(url_seg)
        url = urlunparse((url_seg[2], url_seg[3], url_seg[4], None, url_seg[5], None))
        print(url)

        resp = get(url)
        # 응답이 성공적이고, 내용이 text 또는 html일 때
        if resp.status_code ==  200 and\
           re.search(r'text\/html', resp.headers['content-type']):
              dom = Beautifulsoup(resp.text)
              # dom.select('a[href], iframe[src]')
              for tag in dom.select('a[href], iframe[src]'):
                  link = tag.attrs['href' if tag.has_attr('href') else 'src']
                  newLink = urljoin(url, link)

                  # newlink_seg의 net loc있는지 확인 PK 선택
                  newlink_seg = urlparse(newLink)
                  cur.execute('SELECT PK FROM URLs WHERE NETLOC=?',(newlink_seg.netloc,))
                  temp = cur.fetchone()

############### 여기 부터 다시 보기
                  if not temp:
                      cur.execute('INSERT INTO URLs (SCHEME, NETLOC) VALUES(?,?)', url_seg[:2])
                  
                  cur.execute('SELECT PK FROM URLs WHERE NETLOC=?',(url_seg.netloc,))
                  PK = cur.fetchone()[0]

                  
            

(1, 1, 'https', 'search.naver.com', '/search.naver', 'where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지')
https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지


NameError: name 'Beautifulsoup' is not defined

In [35]:
# URLs 들어간 내용 확인하기
# Links Visited 내용 확인하기

In [60]:
# 1번 도메인의 outbound link의 수

# 1번 도메인의 outbound를 도메인별로 구분한 수(도메인 입장 inbound)
# 레퍼런스가 1인 것을 찾아내서, 각 link의 inbound를 전체 link 수로 나누면 PR(페이지 랭크) 구해진다.

d = .875
PR(B) = (1-d) + d*(B/A)
        .15 + .85      = [0~1]  # 정규화 , 확률
                                # 확률이 나오면 그다음 부터는 계산을 할 수 있다.

# 이렇게 링크들을 수집하면, 
# 링크분석, 
# 지난주 내용을 적용할 수도 있다.